In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Step 1: Load and preprocess the data

In [2]:
# Define paths to training and validation data
train_dir = r'D:\Deep Learning\Text_Image_Language_Classifier\Text Image Dataset\train'
val_dir = r'D:\Deep Learning\Text_Image_Language_Classifier\Text Image Dataset\test'


In [3]:
# Image data generator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255.,  # Rescale pixel values from 0-255 to 0-1
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [4]:
# Validation data generator should only rescale the images
val_datagen = ImageDataGenerator(rescale=1./255.)

In [5]:
# Create generators for training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize images to match the input size of the model
    batch_size=32,
    class_mode='categorical'  # Use categorical labels
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),  # Resize images to match the input size of the model
    batch_size=32,
    class_mode='categorical'  # Use categorical labels
)

Found 10575 images belonging to 12 classes.
Found 3452 images belonging to 12 classes.


# Step 2: Load the ResNet50 model without the top layer

In [6]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD

In [7]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [8]:
# Freeze the layers of ResNet50
for layer in base_model.layers:
    layer.trainable = False

In [9]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(12, activation='softmax'))

In [10]:
model.compile(optimizer=SGD(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)


In [ ]:
resnet_history = model.fit(train_generator, validation_data=val_generator, epochs=10)